In [21]:
import numpy as np
import autograd
import autograd.variable as av 
import autograd.blocks.block as blk
import autograd.blocks.trigo as trig
import autograd.blocks.operations as op

Example 2: `f(x) = sin(x) + cos(x)` for x = 0

In [69]:
#Set the x variable
x = av.Variable(0)

In [70]:
#perform first operation
b1 = trig.sin()(x)

In [71]:
#perform the second operation
b2 = trig.cos()(x)

In [72]:
#perform the final operation 
b3 = op.add()(b1,b2)

In [73]:
print(b2.data)
print(b2.gradient)

[1.]
[[-0.]]


In [54]:
print(b1.data)
print(b1.gradient)

[0.]
[[1.]]


In [67]:
print(b3.data)
print(b3.gradient)

[1.]
[[1.]]


Example 2: `f(x)=sin(cos(x+3)) + e^(sin(x)^2)` for `x = pi`

In [75]:
x = av.Variable(np.pi)

In [79]:
b1 = op.add()(x,3)

AttributeError: 'int' object has no attribute 'data'